### Import thr Dependencies


In [151]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [152]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [153]:
# printing the stopwords in english
print(stopwords.words('English'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Data Pre-processing


In [154]:
# loading the dataset to a pandas datafram
train_df = pd.read_csv('train.csv')

### About the Dataset:

##### 1. id: unique news id for a article

##### 2. title: title of the article

##### 3. author: author of the article

##### 4. Text: text of the article; could be incomplete

##### 5. label: a label that marks whether the news article is real or fack

### 1: Fake News

### 0: real news


In [155]:
train_df.shape

(20800, 5)

In [156]:
#printing first 5 rows of train_df
train_df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \r\nAn Iranian woman has been sentenced ...,1


In [157]:
# counting the number of missing values in train_df
train_df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [158]:
# replacing the null values with empty string
train_df = train_df.fillna('')

In [159]:
# merging the author name and news title

train_df['content'] = train_df['author'] +' ' + train_df['title']

In [160]:
train_df['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [161]:
#separating the data and label
x = train_df.drop('label',axis=1)
y = train_df['label']

In [162]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

### stemming:

##### Stemming is the process of reducing a word to its root word

##### Example:  actor,actress,acting -------->act

In [163]:
port_stem = PorterStemmer()

In [164]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content =' '.join(stemmed_content)
    return stemmed_content


In [165]:
train_df['content'] = train_df['content'].apply(stemming)

In [166]:
train_df['content'].head()

0    darrel lucu hous dem aid even see comey letter...
1    daniel j flynn flynn hillari clinton big woman...
2               consortiumnew com truth might get fire
3    jessica purkiss civilian kill singl us airstri...
4    howard portnoy iranian woman jail fiction unpu...
Name: content, dtype: object

In [167]:
# replacing the data and label
x = train_df['content'].values
y = train_df['label'].values

In [168]:
print(x)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [169]:
# converting the texual data into numerical data
max_features = 9000
vectorizer = TfidfVectorizer(max_features= max_features)
vectorizer.fit(x)

x = vectorizer.transform(x)

In [170]:
print(x)

  (0, 8330)	0.28485063562728646
  (0, 7172)	0.2565896679337957
  (0, 4655)	0.3635963806326075
  (0, 4481)	0.29212514087043684
  (0, 4080)	0.24785219520671603
  (0, 3706)	0.21874169089359144
  (0, 2726)	0.233316966909351
  (0, 2047)	0.2705332480845492
  (0, 1928)	0.3598939188262559
  (0, 1580)	0.2468450128533713
  (0, 1294)	0.3676519686797209
  (0, 145)	0.27010124977708766
  (1, 8846)	0.30071745655510157
  (1, 3605)	0.1904660198296849
  (1, 3014)	0.7143299355715573
  (1, 1910)	0.26373768806048464
  (1, 1494)	0.19094574062359204
  (1, 1165)	0.3827320386859759
  (1, 1006)	0.15521974226349364
  (1, 797)	0.2939891562094648
  (2, 8286)	0.41544962664721613
  (2, 5052)	0.49351492943649944
  (2, 3252)	0.3474613386728292
  (2, 2950)	0.3866530551182615
  (2, 1662)	0.46097489583229645
  :	:
  (20797, 6982)	0.2482526352197606
  (20797, 6540)	0.27263457663336677
  (20797, 6440)	0.24778257724396507
  (20797, 5437)	0.08038079000566466
  (20797, 5033)	0.174553480255222
  (20797, 4996)	0.295420400342031

### splitting the data into train and test set

In [171]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [172]:
x_train.shape

(16640, 9000)

In [173]:
y_train.shape

(16640,)

In [174]:
x_test.shape

(4160, 9000)

In [175]:
y_test.shape

(4160,)

### Training the Model: Logistic Regression

In [176]:
model  = LogisticRegression()

In [197]:
model.fit(x_train,y_train)

LogisticRegression()

### Model Evaluation 


#### Accuracy Score

In [178]:
# accuracy score on the training data
X_train_prediction = model.predict(x_train)

accuracy_score(y_train,X_train_prediction)

0.9853365384615385

In [179]:
# accuracy score on the testing data

X_test_prediction = model.predict(x_test)

accuracy_score(y_test,X_test_prediction)

0.9783653846153846

## FINAL TEST ON TEST.CSV

In [180]:
test_df = pd.read_csv('test.csv')
test_result_df = pd.read_csv('submit.csv')
test_df.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [181]:
test_df.isnull().sum()


id          0
title     122
author    503
text        7
dtype: int64

In [182]:
test_result_df.isnull().sum()

id       0
label    0
dtype: int64

In [183]:
test_df = test_df.fillna('')

In [184]:
test_df.isnull().sum()

id        0
title     0
author    0
text      0
dtype: int64

In [185]:
test_df['content'] = test_df['author'] +' ' + test_df['title']

In [186]:
test_df['content'] = test_df['content'].apply(stemming)

In [187]:
x_final_test = test_df['content']
y_final_test = test_result_df['label']


In [188]:
x_final_test.shape


(5200,)

In [189]:
y_final_test.shape

(5200,)

In [190]:
vectorizer.fit(x_final_test)
x_final_test = vectorizer.transform(x_final_test)

In [191]:
x_final_test.shape

(5200, 9000)

In [198]:
X_test_prediction = model.predict(x_final_test)
accuracy_score(X_test_prediction, y_final_test)

0.5484615384615384